In [0]:
%%bash
pip install librosa

In [1]:
import os
from random import shuffle

import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

import librosa as rosa

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
directory = "drive/My Drive/UE_Proell"
res_type = "kaiser_fast"
n_mfcc = 40

preprocessed = f"{directory}/preprocessed"
os.makedirs(preprocessed, exist_ok=True)

In [4]:
train = pd.read_csv(f"{directory}/train_long.csv", sep=",")
test = pd.read_csv(f"{directory}/test.csv", sep=",")

train.sample(n = 5)

,ID,Class
2850,6909,dog_bark
3549,8531,children_playing
521,1241,siren
2670,6506,children_playing
1759,4301,air_conditioner


In [0]:
def feature_extract(row):
   # function to load files and extract features
   file_name = os.path.join(f"{directory}/data/{row['ID']}.wav")
   if not os.path.exists(file_name):
     print(f"File {file_name} does not exist. Abort...")
     return None, None

   # handle exception to check if there isn't a file which is corrupted
   try:
      # here kaiser_fast is a technique used for faster extraction
      X, sample_rate = rosa.load(file_name, res_type=res_type) 
      # we extract mfcc feature from data
      mfccs = np.mean(rosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=n_mfcc).T, axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", file)
      return None, None
 
   feature = mfccs
   label = row["Class"]
 
   return [feature, label]

In [0]:
# axis = 0 / per column
# axis = 1 / per row

train_temp = train.apply(feature_extract, axis=1)
test_temp = test.apply(feature_extract, axis=1)

In [0]:
train_temp = pd.DataFrame({"label": [row[1] for row in train_temp], "feature": [row[0] for row in train_temp]})
test_temp = pd.DataFrame({"label": [row[1] for row in test_temp], "feature": [row[0] for row in test_temp]})

In [0]:
def encode(data):
  X = np.array(data["feature"].tolist())
  y = np.array(data["label"].tolist())

  lb = LabelEncoder()
  y = np_utils.to_categorical(lb.fit_transform(y))

  return [X, y]

X_train, y_train = encode(train_temp)
X_test, y_test = encode(test_temp)

In [0]:
np.save(f"{preprocessed}/X_train", X_train)
np.save(f"{preprocessed}/X_test", X_test)
np.save(f"{preprocessed}/y_train", y_train)
np.save(f"{preprocessed}/y_test", y_test)